In [1]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import json
import pandas as pd
from nltk import word_tokenize
import gensim 
from gensim.models import Word2Vec
from nltk.stem import PorterStemmer
import re
import csv
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

 
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions
from ibm_watson import NaturalLanguageUnderstandingV1

import logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from spacy.lang.en import English
from spacy.matcher import PhraseMatcher
import spacy
import en_core_web_sm
from pandas.core.common import flatten




In [3]:
csv_file = "comments.csv"
df = pd.read_csv(csv_file)
#df2 = df.groupby(["Permalink"]).sum()

Permalink = df["Permalink"]
Comment = df["Comment"]
#print(Comment)

In [8]:
def remove_stopwords(text):
    stop_words = stopwords.words('english')
    stop_words.extend(["also", "and", "its", "bring", "got", "from", "lot", "rather", "even", "from", "but","that","in","you","who","to", "of", "become", "thats", "looking"])
    words = [w for w in text if w not in stop_words]
    return words

In [10]:
stop_clean = remove_stopwords(Comment)

str_list = list(map(str, stop_clean)) # Converting all datatypes to strings
porter = PorterStemmer()

stemmed = []
for s in str_list:
    stemmed.append(porter.stem(s))

#for x in new_list:
    #print(type(x), x)

tokens = []
for i in stemmed:
    i.lower()
    tokens.append(word_tokenize(i))
#print(tokens)
model = Word2Vec(tokens, min_count=1, size=50, workers=3, window=2, sg=1)

def similarity(word):
    word_H=[word]
    for i in range(10):
        word_H.append((model.most_similar(word)[:10])[i][0])

    return(word_H)
print("Words related to Obama: ", similarity("obama"))

print("Words related to Trump: ", similarity("trump"))

print("Words related to riots: ", similarity("riots"))

print("Words related to china: ", similarity("china"))

print("Words related to massachusetts: ", similarity("massachusetts"))
# Examining some contentious words and their most similar words in the comments section

Words related to Obama:  ['obama', 'texas', 'false', 'liberal', 'local', 'million', 'lying', 'months', 'plus', '2020', 'elderly']
Words related to Trump:  ['trump', 'agenda', 'everyone', 'except', 'him', 'free', 'away', 'die', 'pos', 'next', 'person']
Words related to riots:  ['riots', 'lives', 'together', 'outside', 'parks', 'decision', 'governor', 'cost', 'plan', 'lying', 'mayor']
Words related to china:  ['china', 'soros', 'help', 'covid-19', 'night', 'charge', 'election', 'literally', 'water', 'died', "'"]
Words related to massachusetts:  ['massachusetts', 'peaceably', 'fresno', 'psychos', 'keesters', 'privacy', 'skate', 'exceptions', 'trouble', 'transportation', 'alright']


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


In [22]:
#Printing out the geographical locations only
entities = []
nlp = spacy.load('en_core_web_sm') 
flat = list(flatten(tokens))
for word in flat:
    doc = nlp(word) 
    for ent in doc.ents: 
        #print(ent.text, ent.start_char, ent.end_char, ent.label_) 
        if ent.label_ == "GPE":
            entities.append(ent.text)
print(entities)
# Finding all the locations mentioned
#entities=['england', 'seattle', 'minneapolis', 'miami', 'houston', 'detroit', 'america', 'afghanistan', 'houston', 'houston', 'vietnam', 'seattle', 'u.s.', 'minneapolis', 'sharpton', 'america', 'vietnam', 'chicago', 'vietnam', 'minnesota', 'america', 'baltimore', 'california', 'vietnam', 'baltimore', 'vietnam', 'missouri', 'killdozer', 'colorado', 'michigan', 'california', 'america', 'washington', 'california', 'america', 'california', 'u.s.', 'california', 'chicago', 'chicago', 'chicago', 'chicago', 'chicago', 'chicago', 'chicago', 'illinois', 'michigan', 'iowa', 'chicago', 'chicago', 'chicago', 'chicago', 'chicago', 'chicago', 'chicago', 'soooo', 'chicago', 'chicago', 'wisconsin', 'chicago', 'chicago', 'chicago', 'chicago', 'chicago', 'america', 'america', 'china', 'america', 'america', 'u.s.', 'massachusetts', 'florida', 'america', 'wuhan', 'china', 'china', 'america', 'america', 'washington', 'china', 'china', 'phoenix', 'america', 'america', 'florida', 'china', 'china', 'china', 'america', 'china', 'america', 'china', 'china', 'america', 'yaaay', 'china', 'china', 'chinia', 'america', 'texas', 'virginia', 'florida', 'commiefornia', 'florida', 'wisconsin', 'texas', 'mexico', 'mightmar', 'virginia', 'florida', 'texas', 'texas', 'arlington', 'iowa', 'germany', 'america', 'texas', 'michigan', 'texas', 'wuhan', 'economi', 'wuhan', 'wuhan', 'outta', 'michigan', 'america', 'america', 'america', 'china', 'america', 'italy', 'america', 'america', 'america', 'china', 'texas', 'america', 'california', 'kmiec', 'wuhan', 'wuhan', 'wuhan', 'china', 'wuhan', 'america', 'soooo', 'texas', 'italy', 'america', 'trenton', 'america', 'china', 'texas', 'florida', 'illinois', 'springfield', 'oregon', 'florida', 'america', 'idaho', 'illinois', 'california', 'california', 'china', 'america', 'japan', 'sweden', 'america', 'america', 'delaware', 'florida', 'america', 'america', 'wuhan', 'maryland', 'america', 'calif', 'michigan', 'pennsylvania', 'texas', 'texas', 'boston', 'michigan', 'california', 'minnesota', 'michigan', 'mexico', 'california', 'wisconsin', 'california', 'oklahoma', 'arizona', 'virginia', 'iowa', 'california', 'mexico', 'washington', 'massachusetts', 'washington', 'mexico', 'seattle', 'washington', 'pennsylvania', 'kommiefornia', 'washington', 'montana', 'montana', 'california', 'minnesota', 'minnesota', 'iowa', 'missouri', 'boston', 'ohio', 'california', 'america', 'uk', 'america', 'america', 'bronx', 'michigan', 'michigan', 'chad', 'california', 'france', 'jackwagon', 'turkey', 'wuhan', 'america', 'inconveni', 'america', 'america', 'arizona', 'outta', 'florida', 'arizona', 'georgia', 'georgia', 'mexico', 'commiefornia', 'cuba', 'oregon', 'california', 'commiefornia', 'phoenix', 'california', 'america', 'japan', 'u.s.', 'kenya', 'america', 'america', 'dallas', 'houston', 'mexico', 'america', 'montana', 'america', 'arizona', 'america', 'california', 'america', 'california', 'texas', 'illinois', 'california', 'texas', 'holland', 'sweden', 'tennessee', 'nebbia', 'manyyy', 'america', 'italy', 'america', 'dallas', 'georgia', 'florida', 'outta', 'moscow', 'america', 'wuhan', 'faruci', 'china', 'america', 'northrum', 'california', 'america', 'america', 'america', 'michigan', 'america', 'michigan', 'texas', 'kansas', 'wisconsin', 'kentucky', 'texas', 'california', 'california']

['england', 'seattle', 'minneapolis', 'miami', 'houston', 'detroit', 'america', 'afghanistan', 'houston', 'houston', 'vietnam', 'seattle', 'u.s.', 'minneapolis', 'sharpton', 'america', 'vietnam', 'chicago', 'vietnam', 'minnesota', 'america', 'baltimore', 'california', 'vietnam', 'baltimore', 'vietnam', 'missouri', 'killdozer', 'colorado', 'michigan', 'california', 'america', 'washington', 'california', 'america', 'california', 'u.s.', 'california', 'chicago', 'chicago', 'chicago', 'chicago', 'chicago', 'chicago', 'chicago', 'illinois', 'michigan', 'iowa', 'chicago', 'chicago', 'chicago', 'chicago', 'chicago', 'chicago', 'chicago', 'soooo', 'chicago', 'chicago', 'wisconsin', 'chicago', 'chicago', 'chicago', 'chicago', 'chicago', 'america', 'america', 'china', 'america', 'america', 'u.s.', 'massachusetts', 'florida', 'america', 'wuhan', 'china', 'china', 'america', 'america', 'washington', 'china', 'china', 'phoenix', 'america', 'america', 'florida', 'china', 'china', 'china', 'america',

In [ ]:
csv_file2 = "posts.csv"
df2 = pd.read_csv(csv_file2)

Post = df2["Post"]
#print(Post)

# Looking at the posts here

In [ ]:
stop_clean2 = remove_stopwords(Post)

str_list2 = list(map(str, stop_clean2)) # Converting all datatypes to strings


tokens2 = []
for i in str_list2:
    tokens2.append(word_tokenize(i))
#print(tokens)

def similarity2(word):
    model = Word2Vec(tokens2, min_count=1, size=50, workers=3, window=3, sg=1)
    word_H=[word]
    for i in range(10):
        word_H.append((model.most_similar(word)[:10])[i][0])

    return(word_H)
print("Words most similar to unrest", similarity2("unrest"))